In [13]:
import numpy as np
import pandas as pd
ruta='https://raw.githubusercontent.com/jamc88/Signal-Detection-in-FAERS-Data-Science-in-Pharmacovigilance/refs/heads/main/Ejemplo.csv'


In [14]:
df = pd.read_csv(ruta)
df

,farmaco,evento,a,b,c,d
0,Fármaco_A,Insuficiencia_Hepática,50,950,100,20000
1,Fármaco_B,Mareos,4,196,350,25000
2,Fármaco_C,Evento_Raro,3,150,0,25000
3,Fármaco_D,Cefalea,10,2000,500,15000


In [18]:
def calcular_ror_faers(a, b, c, d):

    # Corrección de Haldane
    if a == 0 or b == 0 or c == 0 or d == 0:
        a, b, c, d = a + 0.5, b + 0.5, c + 0.5, d + 0.5

    # 2. Cálculo del ROR puntual
    ror = (a * d) / (b * c)

    # 3. Cálculo del Error Estándar (SE) del logaritmo natural
    se_ln_ror = np.sqrt((1/a) + (1/b) + (1/c) + (1/d))

    # 4. Cálculo de Intervalos de Confianza (95%)
    z_score = 1.96
    ln_ror = np.log(ror)
    limite_inferior = np.exp(ln_ror - z_score * se_ln_ror)
    limite_superior = np.exp(ln_ror + z_score * se_ln_ror)

    return {
        "ROR": round(ror, 2),
        "IC_95_Inferior": round(limite_inferior, 2),
        "IC_95_Superior": round(limite_superior, 2),
        "Casos_a": a  # este 'a' puede tener el +0.5 si se corrigió
    }

datos = pd.read_csv(ruta)

# iterrows() es el que nos permite ir fila por fila
for indice, fila in datos.iterrows():

    # 1. Sacamos los valores de la fila actual
    nombre_farmaco = fila['farmaco']
    nombre_evento = fila['evento']
    val_a = fila['a']
    val_b = fila['b']
    val_c = fila['c']
    val_d = fila['d']

    print(f"--- Fila {indice}: {nombre_farmaco} vs {nombre_evento} ---")

    resultado = calcular_ror_faers(a=val_a, b=val_b, c=val_c, d=val_d)

    print(f"Resultado del Análisis:")
    print(f"ROR: {resultado['ROR']}")
    print(f"Intervalo de Confianza 95%: [{resultado['IC_95_Inferior']} - {resultado['IC_95_Superior']}]")

    if resultado['IC_95_Inferior'] > 1 and val_a >= 3:
        print("Interpretación: SEÑAL DE SEGURIDAD DETECTADA (Estadísticamente significativa).")
    else:
        print("Interpretación: No cumple criterios de señal.")

    print("\n") #para separar del siguiente fármaco

--- Fila 0: Fármaco_A vs Insuficiencia_Hepática ---
Resultado del Análisis:
ROR: 10.53
Intervalo de Confianza 95%: [7.45 - 14.87]
Interpretación: SEÑAL DE SEGURIDAD DETECTADA (Estadísticamente significativa).


--- Fila 1: Fármaco_B vs Mareos ---
Resultado del Análisis:
ROR: 1.46
Intervalo de Confianza 95%: [0.54 - 3.94]
Interpretación: No cumple criterios de señal.


--- Fila 2: Fármaco_C vs Evento_Raro ---
Resultado del Análisis:
ROR: 1162.81
Intervalo de Confianza 95%: [59.8 - 22610.49]
Interpretación: SEÑAL DE SEGURIDAD DETECTADA (Estadísticamente significativa).


--- Fila 3: Fármaco_D vs Cefalea ---
Resultado del Análisis:
ROR: 0.15
Intervalo de Confianza 95%: [0.08 - 0.28]
Interpretación: No cumple criterios de señal.


